## Code To Find Neighborhoods in Toronto With IBM Watson Studio

In [2]:
import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium

print('Packages installed.')

Packages installed.


In [23]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup 

import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim

# import k-means 
from sklearn.cluster import KMeans
import folium
print('Libraries imported.')

Libraries imported.


In [4]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 
print('Scrapping Done !!!')

Scrapping Done !!!


In [5]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Retrieved !!!')

Data Retrieved !!!


In [6]:
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood


In [7]:
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)

In [9]:
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752420,-79.329242
1,M4A,North York,Victoria Village,43.730600,-79.313265
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
3,M6A,North York,Lawrence Heights,43.723270,-79.451286
4,M6A,North York,Lawrence Manor,43.723270,-79.451286
5,M7A,Downtown Toronto,Queen's Park,43.661150,-79.391715
6,M9A,Queen's Park,Queen's Park,43.662299,-79.528195
7,M1B,Scarborough,Rouge,43.811525,-79.195517
8,M1B,Scarborough,Malvern,43.811525,-79.195517
9,M3B,North York,Don Mills North,43.749055,-79.362227


In [24]:
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


CLIENT_ID = 'OUWWFQHQVOGJRQG4U2RQTAAIA1TII5JOSXQMC1UK25OROL5N' # your Foursquare ID
CLIENT_SECRET = 'IJP4JBWJMMCN4EM52ODXE2NVOLK3DO3ZSSS1WNWDIJ20D0TB' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

The geograpical coordinate of Toronto are 43.653963, -79.387207.
Your credentails:
CLIENT_ID: OUWWFQHQVOGJRQG4U2RQTAAIA1TII5JOSXQMC1UK25OROL5N
CLIENT_SECRET:IJP4JBWJMMCN4EM52ODXE2NVOLK3DO3ZSSS1WNWDIJ20D0TB


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


In [26]:
radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, address, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=OUWWFQHQVOGJRQG4U2RQTAAIA1TII5JOSXQMC1UK25OROL5N&client_secret=IJP4JBWJMMCN4EM52ODXE2NVOLK3DO3ZSSS1WNWDIJ20D0TB&ll=43.653963,-79.387207&v=20180604&query=Toronto, ON&radius=500&limit=100'

In [27]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e439b09dd0f8500295ddfa3'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'query': 'toronto on',
  'totalResults': 58,
  'suggestedBounds': {'ne': {'lat': 43.6584630045, 'lng': -79.38099903084075},
   'sw': {'lat': 43.649462995499995, 'lng': -79.39341496915925}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c05ef964a520a6f620e3',
       'name': 'Nathan Phillips Square',
       'location': {'address': '100 Queen St W',
        'crossStreet': 'at Bay St',
        'lat': 43.65227047322295,
        'lng': -79.383516

In [28]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [29]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Nathan Phillips Square,Plaza,43.652270,-79.383516
1,Downtown Toronto,Neighborhood,43.653232,-79.385296
2,Sansotei Ramen 三草亭,Ramen Restaurant,43.655157,-79.386501
3,Japango,Sushi Restaurant,43.655268,-79.385165
4,Art Gallery of Ontario,Art Gallery,43.654003,-79.392922


In [30]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

56 venues were returned by Foursquare.


In [31]:
## create a function to repeat the same process to all the neighborhoods in Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
toronto_venues = getNearbyVenues(names=neighborhoods,
                                 latitudes=neighbors['Latitude'],
                                 longitudes=neighbors['Longitude'])

Parkwoods
Victoria Village
Harbourfront
Lawrence Heights
Lawrence Manor
Queen's Park
Queen's Park
Rouge
Malvern
Don Mills North

Woodbine Gardens
Parkview Hill
Ryerson

Garden District

Glencairn

Cloverdale

Islington
Martin Grove

Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South

Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens

Eringate

Markland Wood
Old Burnhamthorpe

Guildwood

Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks

Woburn
Leaside
Central Bay Street

Christie

Cedarbrae

Hillcrest Village
Bathurst Manor
Downsview North

Wilson Heights
Thorncliffe Park
Adelaide

King

Richmond

Dovercourt Village
Dufferin

Scarborough Village
Fairview

Henry Farm
Oriole

Northwood Park
York University
East Toronto
Harbourfront East

Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park

Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East

The Danforth West

Riverdal

In [36]:
print(toronto_venues.shape)
toronto_venues.head()

(4716, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75242,-79.329242,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.75242,-79.329242,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.73060,-79.313265,Wigmore Park,43.731023,-79.310771,Park
3,Victoria Village,43.73060,-79.313265,Memories of Africa,43.726602,-79.312427,Grocery Store
4,Victoria Village,43.73060,-79.313265,Guardian Drug,43.730584,-79.307432,Pharmacy


In [37]:
# check how many venues were returned for each neighborhood

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Agincourt,13,13,13,13,13,13
Agincourt North,2,2,2,2,2,2
Albion Gardens,16,16,16,16,16,16
Alderwood,4,4,4,4,4,4
Bathurst Manor,1,1,1,1,1,1
Bathurst Quay,71,71,71,71,71,71
Bayview Village,4,4,4,4,4,4
Beaumond Heights,16,16,16,16,16,16


In [38]:
## find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 263 uniques categories.


In [39]:
## Analyze Each Neighborhood

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
column_list = toronto_onehot.columns.tolist()
column_number = int(column_list.index('Neighborhood'))
column_list = [column_list[column_number]] + column_list[:column_number] + column_list[column_number+1:]
toronto_onehot = toronto_onehot[column_list]

toronto_onehot.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
toronto_onehot.shape

(4716, 263)

In [41]:
## group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Adelaide,0.000000,0.000000,0.00,0.030000,0.0,0.010000,0.000000,0.030000,0.000000,...,0.010000,0.000000,0.000000,0.0000,0.000000,0.010000,0.000000,0.000000,0.000000,0.0
1,Agincourt,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.076923,0.000000,0.000000,0.000000,0.000000,0.0
2,Agincourt North,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,Albion Gardens,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,Alderwood,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
5,Bathurst Manor,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
6,Bathurst Quay,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.014085,0.000000,...,0.000000,0.014085,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.014085,0.0
7,Bayview Village,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
8,Beaumond Heights,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
9,Bedford Park,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [42]:
toronto_grouped.shape

(204, 263)

In [43]:
## print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide
----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.06
2        Hotel  0.05
3   Steakhouse  0.04
4          Gym  0.03


----Agincourt----
                venue  freq
0       Shopping Mall  0.15
1                Pool  0.08
2     Bubble Tea Shop  0.08
3       Grocery Store  0.08
4  Chinese Restaurant  0.08


----Agincourt North----
              venue  freq
0          Pharmacy   0.5
1  Sushi Restaurant   0.5
2    Adult Boutique   0.0
3      Noodle House   0.0
4         Nightclub   0.0


----Albion Gardens
----
                  venue  freq
0         Grocery Store  0.12
1           Auto Garage  0.06
2           Coffee Shop  0.06
3  Fast Food Restaurant  0.06
4        Sandwich Place  0.06


----Alderwood----
                   venue  freq
0      Convenience Store  0.25
1                    Pub  0.25
2  Performing Arts Venue  0.25
3                    Gym  0.25
4            Music Venue  0.00


----Bathurst Manor----
                venue  freq
0         Men's Store

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Hotel,Steakhouse,Asian Restaurant,Burger Joint,Breakfast Spot,Restaurant,Bakery,Japanese Restaurant
1,Agincourt,Shopping Mall,Department Store,Bubble Tea Shop,Chinese Restaurant,Vietnamese Restaurant,Pool,Shanghai Restaurant,Sushi Restaurant,Bakery,Grocery Store
2,Agincourt North,Sushi Restaurant,Pharmacy,Zoo Exhibit,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Field
3,Albion Gardens,Grocery Store,Pizza Place,Fast Food Restaurant,Caribbean Restaurant,Fried Chicken Joint,Sandwich Place,Beer Store,Coffee Shop,Park,Auto Garage
4,Alderwood,Convenience Store,Gym,Pub,Performing Arts Venue,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
5,Bathurst Manor,Men's Store,Food Court,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
6,Bathurst Quay,Coffee Shop,Italian Restaurant,Bar,Café,Restaurant,Pub,Speakeasy,Intersection,Bakery,Park
7,Bayview Village,Construction & Landscaping,Dog Run,Trail,Park,Eastern European Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
8,Beaumond Heights,Grocery Store,Pizza Place,Fast Food Restaurant,Caribbean Restaurant,Fried Chicken Joint,Sandwich Place,Beer Store,Coffee Shop,Park,Auto Garage
9,Bedford Park,Sandwich Place,Italian Restaurant,Coffee Shop,Pet Store,Indian Restaurant,Butcher,Café,Pharmacy,Liquor Store,Sports Club


In [46]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 4, 0, 0, 0, 0], dtype=int32)

In [49]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighbors

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752420,-79.329242,2.0,Food & Drink Shop,Park,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Zoo Exhibit,Dumpling Restaurant
1,M4A,North York,Victoria Village,43.730600,-79.313265,0.0,Park,Grocery Store,Pharmacy,Zoo Exhibit,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farm
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166,0.0,Coffee Shop,Bakery,Theater,Boat or Ferry,Café,Health & Beauty Service,Park,Gastropub,Breakfast Spot,French Restaurant
3,M6A,North York,Lawrence Heights,43.723270,-79.451286,0.0,Clothing Store,Toy / Game Store,Cosmetics Shop,American Restaurant,Sushi Restaurant,Furniture / Home Store,Men's Store,Food Court,Platform,Frozen Yogurt Shop
4,M6A,North York,Lawrence Manor,43.723270,-79.451286,0.0,Clothing Store,Toy / Game Store,Cosmetics Shop,American Restaurant,Sushi Restaurant,Furniture / Home Store,Men's Store,Food Court,Platform,Frozen Yogurt Shop


In [55]:
# create map
map_clusters = folium.Map(location=[43.67635739999999, -79.2930312], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color="red",
        fill=True,
        fill_color="yellow",
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [56]:
## Examine Clusters

## Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,Park,Grocery Store,Pharmacy,Zoo Exhibit,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farm
2,Downtown Toronto,0.0,Coffee Shop,Bakery,Theater,Boat or Ferry,Café,Health & Beauty Service,Park,Gastropub,Breakfast Spot,French Restaurant
3,North York,0.0,Clothing Store,Toy / Game Store,Cosmetics Shop,American Restaurant,Sushi Restaurant,Furniture / Home Store,Men's Store,Food Court,Platform,Frozen Yogurt Shop
4,North York,0.0,Clothing Store,Toy / Game Store,Cosmetics Shop,American Restaurant,Sushi Restaurant,Furniture / Home Store,Men's Store,Food Court,Platform,Frozen Yogurt Shop
5,Downtown Toronto,0.0,Coffee Shop,Café,Pharmacy,Sandwich Place,Park,Italian Restaurant,Indian Restaurant,Burrito Place,Falafel Restaurant,Food Truck
6,Queen's Park,0.0,Coffee Shop,Café,Pharmacy,Sandwich Place,Park,Italian Restaurant,Indian Restaurant,Burrito Place,Falafel Restaurant,Food Truck
7,Scarborough,0.0,Zoo Exhibit,Home Service,Women's Store,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant
8,Scarborough,0.0,Zoo Exhibit,Home Service,Women's Store,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant
9,North York,0.0,Construction & Landscaping,Park,Gas Station,Soccer Field,Burger Joint,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Eastern European Restaurant
10,East York,0.0,Pizza Place,Fast Food Restaurant,Athletics & Sports,Gastropub,Bus Line,Rock Climbing Spot,Bank,Intersection,Breakfast Spot,Café


In [57]:
## Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Etobicoke,1.0,Golf Course,Zoo Exhibit,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Dumpling Restaurant
16,Etobicoke,1.0,Golf Course,Zoo Exhibit,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Dumpling Restaurant
17,Etobicoke,1.0,Golf Course,Zoo Exhibit,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Dumpling Restaurant
18,Etobicoke,1.0,Golf Course,Zoo Exhibit,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Dumpling Restaurant
19,Etobicoke,1.0,Golf Course,Zoo Exhibit,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Dumpling Restaurant


In [58]:
## Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2.0,Food & Drink Shop,Park,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Zoo Exhibit,Dumpling Restaurant
60,Downtown Toronto,2.0,Park,Harbor / Marina,Pier,Zoo Exhibit,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant
61,Downtown Toronto,2.0,Park,Harbor / Marina,Pier,Zoo Exhibit,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant
62,Downtown Toronto,2.0,Park,Harbor / Marina,Pier,Zoo Exhibit,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant
121,Central Toronto,2.0,Park,Zoo Exhibit,Dumpling Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
122,Central Toronto,2.0,Park,Zoo Exhibit,Dumpling Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
123,West Toronto,2.0,Sandwich Place,Park,Zoo Exhibit,Farm,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Dumpling Restaurant
124,West Toronto,2.0,Sandwich Place,Park,Zoo Exhibit,Farm,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Dumpling Restaurant
192,Etobicoke,2.0,Lounge,Park,Zoo Exhibit,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant
193,Etobicoke,2.0,Lounge,Park,Zoo Exhibit,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant


In [59]:
## Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
126,Scarborough,3.0,Auto Garage,Zoo Exhibit,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Dumpling Restaurant
127,Scarborough,3.0,Auto Garage,Zoo Exhibit,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Dumpling Restaurant


In [60]:
## Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,North York,4.0,Men's Store,Food Court,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
45,North York,4.0,Men's Store,Food Court,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
46,North York,4.0,Men's Store,Food Court,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
